In [28]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:

# Challenge 1

# read in the data:
import pandas as pd
df_1 = pd.read_csv('data files/turnstile_190706.txt')
df_2 = pd.read_csv('data files/turnstile_190713.txt')
df_3 = pd.read_csv('data files/turnstile_190720.txt')
df_4 = pd.read_csv('data files/turnstile_190727.txt')
df_5 = pd.read_csv('data files/turnstile_190803.txt')
df_6 = pd.read_csv('data files/turnstile_190810.txt')
df_7 = pd.read_csv('data files/turnstile_190817.txt')
df_8 = pd.read_csv('data files/turnstile_190824.txt')
df_9 = pd.read_csv('data files/turnstile_190831.txt')

#concatenate
df_combined = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9])

In [2]:
# Column name 'EXITS' needs to be formatted to remove extra spaces
print(df_combined.columns)
df_combined.rename(columns={df_combined.columns[10]:'EXITS'}, inplace=True)
print(df_combined.columns)

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')
Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')


In [3]:
# Challenge 2

# What data type did pandas assign values in columns 'Date' and 'Time'?
print(type(df_combined.DATE.iloc[0]), type(df_combined.TIME.iloc[0]))

# Combine 'Date' and 'Time' columns and format into a datetime object
df_combined['DATETIME'] = pd.to_datetime(df_combined['DATE']+ ' ' + df_combined['TIME'])
#df_combined.drop(columns=['DATE','TIME'], inplace=True, axis=1)
print(df_combined.head())

<class 'str'> <class 'str'>
    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  00:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  08:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  12:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  16:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  20:00:00   

      DESC  ENTRIES    EXITS            DATETIME  
0  REGULAR  7116734  2410183 2019-06-29 00:00:00  
1  REGULAR  7116770  2410218 2019-06-29 08:00:00  
2  REGULAR  7116856  2410288 2019-06-29 12:00:00  
3  REGULAR  7117076  2410346 2019-06-29 16:00:00  
4  REGULAR  7117340  2410387 2019-06-29 20:00:00  


Challenge 3

Each row is a turnstile, identified by a combination of the C/A, UNIT, SCP, and STATION columns, with information on entries and exits at that turnstile every n hours. (What is n?) We want total daily entries.
Group the data so that it represents daily entries for each turnstile (hint: pd.groupby or DataFrame.groupby)

In [4]:
df_combined.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,00:00:00,REGULAR,7116734,2410183,2019-06-29 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,08:00:00,REGULAR,7116770,2410218,2019-06-29 08:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,12:00:00,REGULAR,7116856,2410288,2019-06-29 12:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,16:00:00,REGULAR,7117076,2410346,2019-06-29 16:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,20:00:00,REGULAR,7117340,2410387,2019-06-29 20:00:00


In [5]:
# group by turnstile and date, take the first value for each day
turnstiles_daily = (df_combined
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
                        .ENTRIES.first())

In [6]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES
0,A002,R051,02-00-00,59 ST,06/29/2019,7116734
1,A002,R051,02-00-00,59 ST,06/30/2019,7117480
2,A002,R051,02-00-00,59 ST,07/01/2019,7118119
3,A002,R051,02-00-00,59 ST,07/02/2019,7119610
4,A002,R051,02-00-00,59 ST,07/03/2019,7121181


In [7]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .apply(lambda grp: grp.shift(1)))

<ipython-input-7-3a945374269e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily


In [8]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
0,A002,R051,02-00-00,59 ST,06/29/2019,7116734,NaN,NaN
1,A002,R051,02-00-00,59 ST,06/30/2019,7117480,06/29/2019,7116734.0
2,A002,R051,02-00-00,59 ST,07/01/2019,7118119,06/30/2019,7117480.0
3,A002,R051,02-00-00,59 ST,07/02/2019,7119610,07/01/2019,7118119.0
4,A002,R051,02-00-00,59 ST,07/03/2019,7121181,07/02/2019,7119610.0


In [9]:
# remove NaN
turnstiles_daily.dropna(subset=['PREV_DATE'],axis=0,inplace=True)

In [10]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
1,A002,R051,02-00-00,59 ST,06/30/2019,7117480,06/29/2019,7116734.0
2,A002,R051,02-00-00,59 ST,07/01/2019,7118119,06/30/2019,7117480.0
3,A002,R051,02-00-00,59 ST,07/02/2019,7119610,07/01/2019,7118119.0
4,A002,R051,02-00-00,59 ST,07/03/2019,7121181,07/02/2019,7119610.0
5,A002,R051,02-00-00,59 ST,07/04/2019,7122652,07/03/2019,7121181.0


In [11]:
# check which stations have turnstiles with fewer entries than the previous day
(turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
    .size())

C/A   UNIT  SCP       STATION        
A002  R051  02-06-00  59 ST               1
A007  R079  01-05-01  5 AV/59 ST          1
A011  R080  01-03-00  57 ST-7 AV         10
A025  R023  01-03-01  34 ST-HERALD SQ    62
A035  R170  00-00-01  14 ST-UNION SQ      1
                                         ..
R622  R123  00-00-00  FRANKLIN AV        62
R629  R065  00-00-01  ROCKAWAY AV         1
R645  R110  00-03-02  FLATBUSH AV-B.C     2
R646  R110  01-00-01  FLATBUSH AV-B.C    62
R730  R431  00-00-04  EASTCHSTER/DYRE    58
Length: 193, dtype: int64

In [12]:
# If the counter is reversed, make negative values positive
# If difference between days is >= 1 million, set daily entries to min value between entries and prev_ent
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

824 3557655.0
458818 12876202.0
459073 68448066.0
2000091722 6897017.0
11 2029641496.0
7 2997591.0
5327464 4320256.0
4320257 2920966.0
319 5420925.0
721090 5343128.0
262 3407291.0
26 1652502.0
422 17135909.0
22 2129136933.0
21 13593836.0
27 7808518.0
304 4587522.0
1962917508 9137626.0
393224 117441229.0
23 2130567.0
352361657 1409610.0
80 3260287.0
134 3376441.0
351 3839369.0
1487 4721413.0
983271 3670866.0
171 1788082.0
1325342695 11664583.0
105 7542484.0
458765 6627959.0
354 5366386.0
524295 2338090.0
1359886 4065.0
70 5631392.0
871 2543403.0
892 1975856.0
37 1514083866.0
34 12122524.0
25 1229078732.0
1572864 2139.0
1233 4018083.0
214 9789350.0
7 1370664.0
2162762 33622258.0
592416496 389431.0
6 5705107.0
933 1285128.0
1792 6138105.0
1543 10213611.0
18806 654821321.0
163997279 972453993.0
437 36992180.0
78260066 1442544807.0
1862330382 7277226.0
33754095 50516744.0
50474287 33772995.0
1594421086 140471461.0
930 1108422349.0
181 18340503.0
109924748 1928952716.0
98096722 670717696.0
2

In [24]:
turnstiles_daily[turnstiles_daily.values == 12876202]

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
11182,A046,R463,00-06-01,CANAL ST,08/14/2019,12876202,08/13/2019,12873490.0,2712.0
11183,A046,R463,00-06-01,CANAL ST,08/15/2019,458818,08/14/2019,12876202.0,458818.0


In [27]:
mask = ((turnstiles_daily["C/A"] == "A046") &
        (turnstiles_daily["UNIT"] == "R463") & 
        (turnstiles_daily["SCP"] == "00-06-01") & 
        (turnstiles_daily["STATION"] == "CANAL ST"))
turnstiles_daily[mask].tail(20)

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
11179,A046,R463,00-06-01,CANAL ST,08/11/2019,12868628,08/10/2019,12865937.0,2691.0
11180,A046,R463,00-06-01,CANAL ST,08/12/2019,12870812,08/11/2019,12868628.0,2184.0
11181,A046,R463,00-06-01,CANAL ST,08/13/2019,12873490,08/12/2019,12870812.0,2678.0
11182,A046,R463,00-06-01,CANAL ST,08/14/2019,12876202,08/13/2019,12873490.0,2712.0
11183,A046,R463,00-06-01,CANAL ST,08/15/2019,458818,08/14/2019,12876202.0,458818.0
11184,A046,R463,00-06-01,CANAL ST,08/16/2019,722868,08/15/2019,458818.0,264050.0
11185,A046,R463,00-06-01,CANAL ST,08/17/2019,725887,08/16/2019,722868.0,3019.0
11186,A046,R463,00-06-01,CANAL ST,08/18/2019,728580,08/17/2019,725887.0,2693.0
11187,A046,R463,00-06-01,CANAL ST,08/19/2019,730603,08/18/2019,728580.0,2023.0
11188,A046,R463,00-06-01,CANAL ST,08/20/2019,733094,08/19/2019,730603.0,2491.0


In [ ]:
tmp = turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]

In [ ]:
# On what day of the week did users rate the most movies?
turnstiles_daily["dayOfWeek"] = turnstiles_daily.DATE.dt.dayofweek


In [ ]:
moviesPerDayRated = turnstiles_daily.groupby("dayOfWeek").Rating.size()
print(moviesPerDayRated[moviesPerDayRated==moviesPerDayRated.max()])

In [ ]:
# first get daily entries by station
stations_daily = \
(turnstiles_daily.groupby(['STATION','DATE'])['DAILY_ENTRIES'].sum()
                 .reset_index())
stations_daily.head()

In [ ]:
# then get top 10 stations by daily volume 
# (sum across all days is a reasonable way to define this)
top10_stations = \
    (stations_daily.groupby(['STATION'])['DAILY_ENTRIES'].sum()
                   .reset_index()
                   .sort_values(by='DAILY_ENTRIES',ascending=False) 
                   .STATION.head(10))
top10_stations

In [ ]:
df_combined['TURNSTILES'] = (df_combined['STATION'] + '_' + df_combined['C/A'] + '_' + df_combined['UNIT'] + '_' + df_combined['SCP'])

turnstiles_df = turnstiles_df.sort_values(by=['DATETIME'])

turnstiles_df = df_combined.groupby(["TURNSTILES", "DATETIME", "TIME"]).sum().diff(axis=0).fillna(0)

In [ ]:
# FILTER FOR TIME BEFORE COMBINING INTO GROUPS

In [ ]:
df_combined["hour"] = df_combined.DATETIME.dt.hour

In [ ]:
hour_filter = (df_combined['hour'] < 21) & (df_combined['hour'] > 6)
filtered_df = df_combined[hour_filter]

In [ ]:
#filtered_df['TURNSTILES'] = (filtered_df['STATION'] + '_' + filtered_df['C/A'] + '_' + filtered_df['UNIT'] + '_' + filtered_df['SCP'])
#filtered_df = filtered_df.sort_values(by=['C/A','UNIT','SCP','STATION','DATETIME'])
filtered_df = filtered_df.groupby(["STATION","hour"]).sum().fillna(0)





In [ ]:
turnstiles_df.drop(columns=['ENTRIES','EXITS'], inplace=True)

In [ ]:
sample_df = turnstiles_df.sample(1000, random_state=1)

In [ ]:
# find busiest hour within subset of hours (times < 06:00 and times >21:00)
sample_df["hour"] = sample_df.DATETIME.dt.hour


#sample_df[''] = 
#mask = sample_df['DATETIME'] > 06:00
#filtered_df = sample_df[]



In [ ]:
# find busiest hour within subset of hours (times < 06:00 and times >21:00)

# Sum Entries and Exits

# Do we still have high negative and positive counts?
#   are there ways to clean this data?

#
#   Step 1 what are the busiest stations?
#

# group by station, get sum
station_df = df_combined.groupby(["STATION", "DATETIME"]).sum().diff(axis=0).fillna(0)

# group by station, get mean
station_df = df_combined.groupby(["STATION", "DATETIME"]).mean().diff(axis=0).fillna(0)

# look at SD to understand how variable values are at each station



# bar chart - all
# bar chart - top ~30?




#
#   Step 2 what stations are most important to the client?
#

# Census data: residential/ tourist/ income

# Visualize relevant neighborhoods with NYC map

# bar chart - top ~30 highlight/ gray-out

# conclusions about recommended stations for canvasing



#
#   Step 3 break down stations by busiest hour to refine recommendation about when to canvas
#

# focus on the short list of prioritized stations

# write OR statement to include times in 8a-12p and 9a-1p (8a-1p)













